# Методы синхронизации потоков (Лабораторная работа №3)
## Выполнил: студент группы БВТ2001 Хайдаров Адель Фанисович
##### Цель работы: получение практических навыков использования Win32 API для синхронизации потоков.

### Вспомогательные функции

In [2]:
open System.Runtime.InteropServices
open System.Text

type dw = uint
type lps = StringBuilder
type hdl = nativeint

#r "nuget: RayLib-CsLo"

Installed Packages RayLib-CsLo, 4.2.0.3

### Функции Win32 API
Ниже представлены использованные функции Win32 API для получения той или иной информации о системе и для задания некоторых параметров

In [3]:
[<DllImport("kernel32.dll")>] extern dw GetCurrentThreadId()
[<DllImport("kernel32.dll")>] extern dw GetCurrentProcessId()
[<DllImport("kernel32.dll")>] extern void Sleep(dw ms)

type CriticalSection = struct
    val debug: int64
    val lockCount: int; val recursionCount: int
    val owningThread: hdl; val lockSemaphore: hdl
    val spinCount: dw
end

[<DllImport("kernel32.dll")>] extern void InitializeCriticalSection(CriticalSection& s)
[<DllImport("kernel32.dll")>] extern void EnterCriticalSection(CriticalSection& s)
[<DllImport("kernel32.dll")>] extern void LeaveCriticalSection(CriticalSection& s)
[<DllImport("kernel32.dll")>] extern void DeleteCriticalSection(CriticalSection& s)

let INFINITE = 0xFFFFFFFFu

type WState = | Abandoned = 0x80 | TimedOut = 0x102 | Signaled = 0
type Access = | All = 0x1F0001

[<DllImport("kernel32.dll")>] extern WState WaitForSingleObject(hdl handle, dw ms)

[<DllImport("kernel32.dll")>] extern hdl CreateMutex(string sec, bool isInitial, string name)
[<DllImport("kernel32.dll")>] extern hdl OpenMutex(Access access, bool isInitial, string name)
[<DllImport("kernel32.dll")>] extern bool ReleaseMutex(hdl handle)

[<DllImport("kernel32.dll")>] extern hdl CreateSemaphore(string l0, int init, int max, string name)
[<DllImport("kernel32.dll")>] extern bool ReleaseSemaphore(hdl sm, int init, int& prev)

[<DllImport("kernel32.dll")>] extern bool CloseHandle(hdl object)

### Выполнение

#### Метод критических секций

In [4]:
let mutable section = CriticalSection()

let mutable cnt = 0

let inc =
    async {
        EnterCriticalSection &section
        for _ = 1 to 10000 do cnt <- cnt + 1
        LeaveCriticalSection &section
    }

let racingInc =
    async { for _ = 1 to 10000 do cnt <- cnt + 1 }
    
let run (abc: Async<'s>) times = 
    InitializeCriticalSection &section
    seq { for _ = 1 to times do abc } |> Async.Parallel |> Async.RunSynchronously |> ignore
    DeleteCriticalSection &section
    cnt

display $"Правильное выполнение: {run inc 3}"
display $"Неправильное выполнение: {run racingInc 3}"

Правильное выполнение: 30000

Неправильное выполнение: 50855

#### Метод мьютексов

Поток машин на ж/д переезде:

In [5]:
display $"ID процесса: {GetCurrentProcessId()}"

let mutex: hdl = OpenMutex(Access.All, false, "MX1")

if mutex = 0 then display "Шлагбаума нет"
else display "Ожидание..."

let wait = WaitForSingleObject(mutex, INFINITE)
if mutex <> 0 then display "Шлагбаум открыт" |> ignore

for t = 1 to 20 do printf $"{t} "; Sleep 500u

ReleaseMutex mutex

display "Все проехали переезд"

CloseHandle mutex |> ignore

ID процесса: 18108

Ожидание...

Шлагбаум открыт

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

Все проехали переезд

Шлагбаум (запускать отдельным приложением):

In [ ]:
open Raylib_CsLo
open type Raylib
open type KeyboardKey

let pid = GetCurrentProcessId()
let mutex = CreateMutex(null, false, "MX1")

let width, height = 400, 500
InitWindow(width, height, "Шлагбаум"); SetTargetFPS 60

let btnX, btnY, btnW, btnH = 10, 35, width - 20, height - 45

let mutable pause, complete = true, false

WaitForSingleObject(mutex, INFINITE) |> ignore

while WindowShouldClose() |> not do
    if IsKeyPressed KEY_SPACE then
        if pause then ReleaseMutex mutex |> ignore
        if complete |> not then pause <- not pause

        complete <- true

    let status = if pause then "STOP!" else "GO!"
    let tW, tH = MeasureText(status, 40), 40

    BeginDrawing(); ClearBackground RAYWHITE

    DrawRectangle(btnX, btnY, btnW, btnH, (if pause then Color(r = 255, g = 0, b = 30, a = 200) else SKYBLUE))
    DrawText($"Process ID: {pid}", 10, 10, 20, DARKGRAY)
    DrawText(status, btnX + (btnW - tW) / 2, btnY + (btnH - tH) / 2, 40, DARKGRAY)

    EndDrawing()

CloseHandle mutex |> ignore
CloseWindow()

#### Метод семафоров

In [ ]:
let sm = CreateSemaphore(null, 4, 4, "CarParking")

display "Парковка открыта на 30 секунд"

Sleep 30000u

CloseHandle sm |> ignore

Парковка открыта на 30 секунд

# Вывод

В результате лабораторной работы мной была проведена работа с потоками и процессами, используя средства Win32 API.